In [60]:
import pandas as pd
import numpy as np
from scipy import stats as st


# Auxiliary functions 

In [61]:
def fit_gamma_mle(data):
    """
    Fit gamma distribution using MLE
    Returns: shape_mle, loc_mle, scale_mle, rate_mle
    """
    shape_mle, loc_mle, scale_mle = st.gamma.fit(data, floc=0)
    rate_mle = 1 / scale_mle
    return shape_mle, loc_mle, scale_mle, rate_mle

def lrt_gamma(YA, YB, k_hat):
    nA = len(YA)
    nB = len(YB)
    YbarA = np.mean(YA)
    YbarB = np.mean(YB)
    Ybar = (nA * YbarA + nB * YbarB) / (nA + nB)

    return 2 * k_hat * (
        nA * np.log(Ybar / YbarA) +
        nB * np.log(Ybar / YbarB)
    )

def lrt_normal(YA, YB, sigma2):
    nA = len(YA)
    nB = len(YB)

    meanA = np.mean(YA)
    meanB = np.mean(YB)

    return (1 / sigma2) * (nA * nB / (nA + nB)) * (meanA - meanB)**2


In [62]:
df = pd.read_csv('ndt_tests_corrigido.csv')

df.head()

,timestamp,download_throughput_bps,rtt_download_sec,upload_throughput_bps,rtt_upload_sec,packet_loss_percent,client,server
0,2025-08-09 15:28:02.000 +0000,8.223657e+07,0.231862,6.973217e+07,0.247727,0.000000,client12,server06
1,2025-08-09 15:30:11.000 +0000,9.027315e+08,0.012000,8.521780e+08,0.005423,0.008226,client01,server07
2,2025-08-10 04:27:43.000 +0000,5.910655e+08,0.014000,2.812188e+08,0.014544,5.954284,client13,server07
3,2025-08-09 22:45:07.000 +0000,6.721139e+08,0.011000,1.135400e+08,0.010482,0.261528,client12,server07
4,2025-08-10 04:49:21.000 +0000,8.122087e+08,0.009000,6.857905e+08,0.009368,1.381646,client03,server03


# Analyzing the data


In [63]:
df.describe()

,download_throughput_bps,rtt_download_sec,upload_throughput_bps,rtt_upload_sec,packet_loss_percent
count,7.087000e+03,7087.000000,7.087000e+03,7087.000000,7087.000000
mean,5.062996e+08,0.048447,3.781607e+08,0.037194,1.899137
std,3.352129e+08,0.067859,3.014096e+08,0.063537,3.449941
min,-1.000000e+00,-0.001000,-1.000000e+00,-1.000000,-1.000000
25%,1.642170e+08,0.008000,9.733497e+07,0.004918,0.001484
50%,5.960927e+08,0.011549,2.979242e+08,0.010858,0.189771
75%,8.489080e+08,0.115000,6.701694e+08,0.017723,2.050781
max,9.332671e+08,0.398051,9.214500e+08,0.360227,22.389090


In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7087 entries, 0 to 7086
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   timestamp                7087 non-null   object 
 1   download_throughput_bps  7087 non-null   float64
 2   rtt_download_sec         7087 non-null   float64
 3   upload_throughput_bps    7087 non-null   float64
 4   rtt_upload_sec           7087 non-null   float64
 5   packet_loss_percent      7087 non-null   float64
 6   client                   7087 non-null   object 
 7   server                   7087 non-null   object 
dtypes: float64(5), object(3)
memory usage: 443.1+ KB


# Data Preprocessing


In [65]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

metrics = [
    'download_throughput_bps',
    'rtt_download_sec',
    'upload_throughput_bps',
    'rtt_upload_sec',
    'packet_loss_percent'
]

# Clean the DataFrame
final_df = df.dropna(subset=metrics)
final_df = final_df[(final_df[metrics] >= 0).all(axis=1)]

# Transform rtt to ms
final_df['rtt_download_ms'] = final_df['rtt_download_sec'] * 1000
final_df['rtt_upload_ms'] = final_df['rtt_upload_sec'] * 1000

metrics = [
    'log_download_throughput',
    'log_upload_throughput',
    'packet_loss_percent',
    'rtt_download_ms',
    'rtt_upload_ms',
]


In [66]:
df_group_client = final_df.groupby('client')
df_group_server = final_df.groupby('server')

In [67]:
client_05 = final_df[final_df['client'] == 'client05']
server_04 = final_df[final_df['server'] == 'server04']

# SEGUNDA PARTE DO TRABALHO: TESTE DE HIPÓTESE


Para as variáveis de throughput (download e upload) e RTT (download e upload), realizamos testes de hipótese para comparar as médias entre os conjuntos de dados client_05 e server_04.

Para isso, começaremos definindo as hipóteses nula e alternativa para cada métrica:
- Hipótese Nula (H0): As médias das métricas são iguais entre client_05 e server_04.
- Hipótese Alternativa (H1): As médias das métricas são diferentes entre client_05 e server_04.

Para calcular a verossimilhança sob H0, combinamos os dados dos dois conjuntos e ajustamos o modelo estatístico apropriado (Normal para RTT e Gamma para throughput) aos dados combinados. Calculamos a verossimilhança dos dados observados sob este modelo combinado.

Para calcular a verossimilhança sob H1, ajustamos o modelo estatístico separadamente para cada conjunto de dados (client_05 e server_04) e calculamos a verossimilhança dos dados observados sob esses modelos separados.



## Calculando a verossimilhança sob H0 e H1 para throughput de download

In [68]:

YA_download_throughput = client_05['download_throughput_bps']
YB_download_throughput = server_04['download_throughput_bps']
Y_download_throughput_concatenado = np.concatenate([YA_download_throughput, YB_download_throughput])

k_download_throughput_chapeu, _, _, _ = fit_gamma_mle(Y_download_throughput_concatenado)
lrt_statistic_download_throughput = lrt_gamma(YA_download_throughput, YB_download_throughput, k_download_throughput_chapeu)
print(f"Média do Throughput de Download para o Cliente 05: {np.mean(YA_download_throughput):.2f} bps")
print(f"Média do Throughput de Download para o Servidor 04: {np.mean(YB_download_throughput):.2f} bps")
print(f"Parâmetro k: {k_download_throughput_chapeu:.6f}")
print(f"Estatística LRT para o Throughput de Download entre o Cliente 05 e o Servidor 04: {lrt_statistic_download_throughput:.4f}")


Média do Throughput de Download para o Cliente 05: 309530969.17 bps
Média do Throughput de Download para o Servidor 04: 196923197.80 bps
Parâmetro k: 1.346985
Estatística LRT para o Throughput de Download entre o Cliente 05 e o Servidor 04: 72.7815


## Calculando a verossimilhança sob H0 e H1 para throughput de upload

In [69]:

YA_upload_throughput = client_05['upload_throughput_bps']
YB_upload_throughput = server_04['upload_throughput_bps']
Y_upload_throughput_concatenado = np.concatenate([YA_upload_throughput, YB_upload_throughput])

k_upload_throughput_chapeu, _, _, _ = fit_gamma_mle(Y_upload_throughput_concatenado)
lrt_statistic_upload_throughput = lrt_gamma(YA_upload_throughput, YB_upload_throughput, k_upload_throughput_chapeu)
print(f"Média do Throughput de Upload para o Cliente 05: {np.mean(YA_upload_throughput):.2f} bps")
print(f"Média do Throughput de Upload para o Servidor 04: {np.mean(YB_upload_throughput):.2f} bps")
print(f"Parâmetro k: {k_upload_throughput_chapeu:.6f}")
print(f"Estatística LRT para o Throughput de Upload entre o Cliente 05 e o Servidor 04: {lrt_statistic_upload_throughput:.4f}")


Média do Throughput de Upload para o Cliente 05: 347752207.84 bps
Média do Throughput de Upload para o Servidor 04: 157294521.67 bps
Parâmetro k: 1.237391
Estatística LRT para o Throughput de Upload entre o Cliente 05 e o Servidor 04: 204.2865


Iremos calcular o p-valor associado ao teste de hipótese utilizando a razão de verossimilhança (LR):
LR = L(H1) / L(H0)

Um p-valor baixo (geralmente < 0.05) indicaria evidências suficientes para rejeitar a hipótese nula em favor da hipótese alternativa, sugerindo que as médias das métricas são significativamente diferentes entre os dois conjuntos de dados. Usaremos alpha = 0.05 como nível de significância para nossos testes.

In [70]:
p_value_download_throughput = st.chi2.sf(lrt_statistic_download_throughput, df=1)
p_value_upload_throughput = st.chi2.sf(lrt_statistic_upload_throughput, df=1)

print(f"P-valor para o Throughput de Download (LRT): {p_value_download_throughput:.3e}")
print(f"P-valor para o Throughput de Upload (LRT): {p_value_upload_throughput:.3e}")

P-valor para o Throughput de Download (LRT): 1.448e-17
P-valor para o Throughput de Upload (LRT): 2.424e-46


Como p-valor para ambos throughput de download e upload são menores que 0.05, rejeitamos a hipótese nula para ambas as métricas. Isso indica que há evidências estatisticamente significativas de que as médias de throughput de download e upload diferem entre os conjuntos de dados client_05 e server_04.

## Calculando a verossimilhança sob H0 e H1 para RTT de download

In [71]:
YA_download_rtt = client_05['rtt_download_ms']
YB_download_rtt = server_04['rtt_download_ms']
Y_download_rtt_concatenado = np.concatenate([YA_download_rtt, YB_download_rtt])

sigma2_download_rtt_chapeu = np.mean((Y_download_rtt_concatenado - np.mean(Y_download_rtt_concatenado))**2)

lrt_statistic_download_rtt = lrt_normal(YA_download_rtt, YB_download_rtt, sigma2_download_rtt_chapeu)
print(f"Média do RTT de Download para o Cliente 05: {np.mean(YA_download_rtt):.2f} ms")
print(f"Média do RTT de Download para o Servidor 04: {np.mean(YB_download_rtt):.2f} ms")
print(f"Variância estimada do RTT de Download conjunto: {sigma2_download_rtt_chapeu:.4f} ms²")
print(f"Estatística LRT para o RTT de Download entre o Cliente 05 e o Servidor 04: {lrt_statistic_download_rtt:.4f}")

Média do RTT de Download para o Cliente 05: 45.17 ms
Média do RTT de Download para o Servidor 04: 127.81 ms
Variância estimada do RTT de Download conjunto: 5475.6416 ms²
Estatística LRT para o RTT de Download entre o Cliente 05 e o Servidor 04: 327.8311


In [72]:

YA_upload_rtt = client_05['rtt_upload_ms']
YB_upload_rtt = server_04['rtt_upload_ms']
Y_upload_rtt_concatenado = np.concatenate([YA_upload_rtt, YB_upload_rtt])

sigma2_upload_rtt_chapeu = np.mean((Y_upload_rtt_concatenado - np.mean(Y_upload_rtt_concatenado))**2)

lrt_statistic_upload_rtt = lrt_normal(YA_upload_rtt, YB_upload_rtt, sigma2_upload_rtt_chapeu)
print(f"Média do RTT de Upload para o Cliente 05: {np.mean(YA_upload_rtt):.2f} ms")
print(f"Média do RTT de Upload para o Servidor 04: {np.mean(YB_upload_rtt):.2f} ms")
print(f"Variância estimada do RTT de Upload conjunto: {sigma2_upload_rtt_chapeu:.4f} ms²")
print(f"Estatística LRT para o RTT de Upload entre o Cliente 05 e o Servidor 04: {lrt_statistic_upload_rtt:.4f}")

Média do RTT de Upload para o Cliente 05: 29.71 ms
Média do RTT de Upload para o Servidor 04: 129.14 ms
Variância estimada do RTT de Upload conjunto: 5972.2086 ms²
Estatística LRT para o RTT de Upload entre o Cliente 05 e o Servidor 04: 435.1059


In [73]:
p_value_download_rtt = st.chi2.sf(lrt_statistic_download_rtt, df=1)
p_value_upload_rtt = st.chi2.sf(lrt_statistic_upload_rtt, df=1)

print(f"P-valor para o RTT de Download (LRT): {p_value_download_rtt:.3e}")
print(f"P-valor para o RTT de Upload (LRT): {p_value_upload_rtt:.3e}")

P-valor para o RTT de Download (LRT): 2.852e-73
P-valor para o RTT de Upload (LRT): 1.258e-96


Novamente, como os p-valores para RTT de download e upload são menores que 0.05, rejeitamos a hipótese nula para ambas as métricas. Isso indica que há evidências estatisticamente significativas de que as médias de RTT de download e upload diferem entre os conjuntos de dados client_05 e server_04 sob o modelo Normal com variância comum.

Como alternativa equivalente ao critério do p-valor, pode-se comparar a estatística do teste com o valor crítico da distribuição chi-squared com 1 grau de liberdade (3.841). Em todos os casos analisados, as estatísticas LRT observadas são muito superiores a esse valor crítico, levando à rejeição da hipótese nula.